In [12]:
#Paper implementation

In [18]:
import os
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import imageio
from albumentations import VerticalFlip, HorizontalFlip, ElasticTransform, GridDistortion, OpticalDistortion, CoarseDropout, ShiftScaleRotate, Compose, RandomRotate90

In [19]:
# takes a path and creates a directory if it does not exist
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [20]:
def load_data(path):
    """X = images and Y = masks """
    train_x = sorted(glob(os.path.join(path, "Train", "Image", "*.*")))
    train_y = sorted(glob(os.path.join(path, "Train", "Mask", "*.*")))

    test_x = sorted(glob(os.path.join(path, "Test", "Image", "*.*")))
    test_y = sorted(glob(os.path.join(path, "Test", "Mask", "*.*"))) 
    return (train_x, train_y), (test_x, test_y)

In [21]:
def augment_data(images, masks, save_path, augment=True):
    H = 512
    W = 512

    for idx, (x, y) in tqdm(enumerate(zip(images, masks)), total=len(images)):
        """Extracting names"""
        name = x.split("\\")[-1].split(".")[0]

        """Reading image and mask"""
        x = cv2.imread(x, cv2.IMREAD_COLOR)
        y = imageio.imread(y)
        y = np.array(y)

        if augment:
            
            aug = RandomRotate90(p=1.0)
            augmented = aug(image=x, mask=y)
            x4 = augmented['image']
            y4 = augmented['mask']
            
            aug = RandomRotate90(p=1.0)
            augmented = aug(image=x4, mask=y4)
            x5 = augmented['image']
            y5 = augmented['mask']
            
            aug = RandomRotate90(p=1.0)
            augmented = aug(image=x5, mask=y5)
            x6 = augmented['image']
            y6 = augmented['mask']
            
#Horizontal Flip
            aug = HorizontalFlip(p=1.0)
            augmented = aug(image=x, mask=y)
            x1 = augmented["image"]
            y1 = augmented["mask"]
            
            aug = RandomRotate90(p=1.0)
            augmented = aug(image=x1, mask=y1)
            x7 = augmented['image']
            y7 = augmented['mask']
            
            aug = RandomRotate90(p=1.0)
            augmented = aug(image=x7, mask=y7)
            x8 = augmented['image']
            y8 = augmented['mask']
            
            aug = RandomRotate90(p=1.0)
            augmented = aug(image=x8, mask=y8)
            x9 = augmented['image']
            y9 = augmented['mask']
            
#Vertical Flip
            aug = VerticalFlip(p=1.0)
            augmented = aug(image=x, mask=y)
            x2 = augmented["image"]
            y2 = augmented["mask"]
            
            aug = RandomRotate90(p=1.0)
            augmented = aug(image=x2, mask=y2)
            x10 = augmented['image']
            y10 = augmented['mask']
            
            aug = RandomRotate90(p=1.0)
            augmented = aug(image=x10, mask=y10)
            x11 = augmented['image']
            y11 = augmented['mask']
            
            aug = RandomRotate90(p=1.0)
            augmented = aug(image=x11, mask=y11)
            x12 = augmented['image']
            y12 = augmented['mask']

#Only X-Y translation
            aug = Compose([ShiftScaleRotate (rotate_limit=0, border_mode=cv2.BORDER_CONSTANT, shift_limit_x=-0.05, shift_limit_y=0.05, p=1)])
            augmented = aug(image=x, mask=y)
            x3 = augmented['image']
            y3 = augmented['mask']
            
            

            X = [x, x1, x2, x4, x5, x6, x7, x8, x9, x10, x11, x12]
            Y = [y, y1, y2, y4, y5, y6, y7, y8, y9, y10, y11, y12]

        else:
            X = [x]
            Y = [y]

        index = 0
        for image, mask in zip(X, Y):

            if len(X) == 1:
                tmp_image_name = f"{name}.jpg"
                tmp_mask_name = f"{name}.jpg"
            else:
                tmp_image_name = f"{name}_{index}.jpg"
                tmp_mask_name = f"{name}_{index}.jpg"

            image_path = os.path.join(save_path, "Image", tmp_image_name)
            mask_path = os.path.join(save_path, "Mask", tmp_mask_name)

            """Saving the image"""
            cv2.imwrite(image_path, image)
            cv2.imwrite(mask_path, mask)

            index += 1

In [22]:
if __name__ == "__main__":
    """Seeding"""
    np.random.seed(42)

    """Load the data"""
    data_path = "Datasets/DRIVE/Drive_enhanced/"
    (train_x, train_y), (test_x, test_y) = load_data(data_path)
    print(f"Train: {len(train_x)} - {len(train_y)}")
    
    """DATA AUGMENTATION"""
    # Creating directories
    create_dir("Datasets/DRIVE/Drive_Aug/Train/image")
    create_dir("Datasets/DRIVE/Drive_Aug/Train/Mask")
    create_dir("Datasets/DRIVE/Drive_Aug/Test/Image")
    create_dir("Datasets/DRIVE/Drive_Aug/Test/Mask")

    augment_data(train_x, train_y, "Datasets/DRIVE/Drive_Aug/Train/", augment=True)
    augment_data(test_x, test_y, "Datasets/DRIVE/Drive_Aug/Test/", augment=True)

Train: 20 - 20


  0%|          | 0/20 [00:00<?, ?it/s]C:\Users\User\AppData\Local\Temp\ipykernel_19944\2951491552.py:11: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  y = imageio.imread(y)
100%|██████████| 20/20 [00:03<00:00,  5.57it/s]
